In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import argparse
from prettytable import PrettyTable

In [3]:
from sampling import *

import torch.utils.data as data
from torch.utils.data import DataLoader

In [4]:
import torch
import torch.optim as optim

In [5]:
import os
import torch
import pickle
from tqdm import tqdm

In [6]:
def print_args(parse_args):
    x = PrettyTable()
    x.field_names = ["Arg.", "Value"]
    for arg in vars(parse_args):
        x.add_row([arg, getattr(parse_args, arg)])
    print(x)

In [7]:
def arg_parse():
        parser = argparse.ArgumentParser(description='LiveRec - Douyu')

        parser.add_argument('--seed', dest='seed', type=int,
            help='Random seed')

        parser.add_argument('--batch_size', dest='batch_size', type=int,
            help='Batch size - only active if torch is used')
        parser.add_argument('--seq_len', dest='seq_len', type=int,
            help='Max size of the sequence to consider')

        parser.add_argument('--num_heads', dest='num_heads', type=int,
            help='Numer of heads to use for multi-heads attention')
        parser.add_argument('--num_heads_ctx', dest='num_heads_ctx', type=int,
            help='Numer of heads to use for multi-heads attention CTX')


        parser.add_argument('--dataset', dest='dataset', 
            help='Input dataset.')
 
        parser.add_argument('--model', dest='model', type=str,
            help='Type of the model')

        parser.add_argument('--model_from', dest='mfrom', type=str,
            help='Name of the model to load')
        parser.add_argument('--model_to', dest='mto', type=str,
            help='Name of the model to save')
        parser.add_argument('--cache_dir', dest='cache_dir', type=str,
            help='Path to save the cached preprocessd dataset')
 
        parser.add_argument('--model_path', dest='model_path', type=str,
            help='Path to save the model')
        parser.add_argument('--early_stop', dest='early_stop', type=int,
            help='Number of iteration without improvment before stop')
        parser.add_argument('--ev_sample', dest='ev_sample', type=int,
            help='Number of samples for the final evaluation')
        parser.add_argument('--device', dest='device', type=str,
            help='Pytorch device')

        parser.add_argument('--lr', dest='lr', type=float,
            help='Learning rate.')
        parser.add_argument('--mask_prob', dest='mask_prob', type=float,
            help='BERT mask prob.')
        parser.add_argument('--l2', dest='l2', type=float,
            help='Strength of L2 regularization')
        parser.add_argument('--dim', dest='K', type=int,
            help='Number of latent factors')
 
        parser.add_argument('--num_iters', dest='num_iter', type=int,
            help='Number of training iterations')
        parser.add_argument('--num_epochs', dest='num_epochs', type=int,
            help='Number of training epochs')
        parser.add_argument('--num_att', dest='num_att', type=int,
            help='Num attention module for seq encoding')
        parser.add_argument('--num_att_ctx', dest='num_att_ctx', type=int,
            help='Num attention for ctx module')
 
        parser.add_argument('--topk_att', dest='topk_att', type=int,
            help='Items to send to attentive output')

        parser.add_argument('--fr_ctx', dest='fr_ctx', nargs='?',
            const=True, default=True,
            help='')
        parser.add_argument('--fr_rep', dest='fr_rep', nargs='?',
            const=True, default=False,
            help='')
        parser.add_argument('--uniform', dest='uniform', nargs='?',
            const=True, default=False,
            help='')
        parser.add_argument('--debug', dest='debug', nargs='?',
            const=True, default=False,
            help='')
#         parser.add_argument('--caching', dest='caching', nargs='?',
#             const=True, default=False,
#             help='')
        parser.add_argument('--caching', dest='caching', nargs='?',
            const=True, default=True,
            help='')

        parser.set_defaults(
                        seed=42,
                        dataset="",
                        lr=0.0005,
                        l2=0.1,  
                        mask_prob=0.5,  
                        batch_size=100, 
                        num_att=2,
                        num_att_ctx=2,
                        num_heads=4,
                        num_heads_ctx=4,
                        num_iter=200,
                        seq_len=16,  
                        topk_att=64,  
                        early_stop=15,  
                        K=64,  
                        num_epochs=150,
#                         model="LiveRec",
                        model="POP",
                        model_path="",
                        mto="liverec",
                        device="cuda",
                        cache_dir=""
                       )


        args, unknown = parser.parse_known_args()
        return args 

In [8]:
args = arg_parse()
print_args(args)
args.device = torch.device(args.device)

+---------------+---------+
|      Arg.     |  Value  |
+---------------+---------+
|      seed     |    42   |
|   batch_size  |   100   |
|    seq_len    |    16   |
|   num_heads   |    4    |
| num_heads_ctx |    4    |
|    dataset    |         |
|     model     |   POP   |
|     mfrom     |   None  |
|      mto      | liverec |
|   cache_dir   |         |
|   model_path  |         |
|   early_stop  |    15   |
|   ev_sample   |   None  |
|     device    |   cuda  |
|       lr      |  0.0005 |
|   mask_prob   |   0.5   |
|       l2      |   0.1   |
|       K       |    64   |
|    num_iter   |   200   |
|   num_epochs  |   150   |
|    num_att    |    2    |
|  num_att_ctx  |    2    |
|    topk_att   |    64   |
|     fr_ctx    |   True  |
|     fr_rep    |  False  |
|    uniform    |  False  |
|     debug     |  False  |
|    caching    |   True  |
+---------------+---------+


In [9]:
INFILE = os.path.join(args.dataset,'douyu_10k_min_duration.csv')
#user,streamer_id,start,stop
cols = ["user","streamer","start","stop","duration"]
data_fu = pd.read_csv(INFILE, header=None, names=cols)
    
# Add one for padding
data_fu.user = pd.factorize(data_fu.user)[0]+1
data_fu['streamer_raw'] = data_fu.streamer
data_fu.streamer = pd.factorize(data_fu.streamer)[0]+1
print("Num users: ", data_fu.user.nunique())
print("Num streamers: ", data_fu.streamer.nunique())
print("Num interactions: ", len(data_fu))

Num users:  11610
Num streamers:  47612
Num interactions:  5378125


In [10]:
data_fu.start.min(),data_fu.start.max(),data_fu.stop.min(),data_fu.stop.max()

(0, 87839, 1, 87840)

In [11]:
args.M = data_fu.user.max()+1 # users
args.N = data_fu.streamer.max()+2 # items
args.D = data_fu.duration.max()+1 # duration

data_temp = data_fu.drop_duplicates(subset=['streamer','streamer_raw'])
umap      = dict(zip(data_temp.streamer_raw.tolist(),data_temp.streamer.tolist()))

In [12]:
args.M ,args.N ,args.D

(11611, 47614, 2883)

In [13]:
# Splitting and caching
max_step = max(data_fu.start.max(),data_fu.stop.max())
print("Num timesteps: ", max_step)
args.max_step = max_step
args.pivot_1  = max_step-500
args.pivot_2  = max_step-250

Num timesteps:  87840


In [14]:
print("caching availability")
ts = {}
max_avail = 0
for s in range(max_step):
    all_av = data_fu[(data_fu.start<=s) & (data_fu.stop>s)].streamer.unique().tolist()
    ts[s] = all_av
    max_avail = max(max_avail,len(ts[s]))
args.max_avail = max_avail
args.ts = ts
print("max_avail: ", max_avail)

caching availability
max_avail:  446


In [15]:
# Compute availability matrix of size (num_timesteps x max_available)
max_av   = max([len(v) for k,v in args.ts.items()])
max_step = max([k for k,v in args.ts.items()])+1
av_tens = torch.zeros(max_step,max_av).type(torch.long)
for k,v in args.ts.items():
    av_tens[k,:len(v)] = torch.LongTensor(v)
args.av_tens = av_tens.to(args.device)

In [16]:
data_fu

,user,streamer,start,stop,duration,streamer_raw
0,1,1,81824,81825,2,1211089da67
1,1,2,73720,73721,1,eb5bf69ab60
2,1,1,81825,81834,18,1211089da67
3,1,2,73673,73676,4,eb5bf69ab60
4,1,1,81745,81750,9,1211089da67
...,...,...,...,...,...,...
5378120,11610,8849,21366,21368,2,80ec36cddc2
5378121,11610,8849,21364,21367,4,80ec36cddc2
5378122,11610,8849,21319,21332,24,80ec36cddc2
5378123,11610,8849,21362,21365,4,80ec36cddc2


In [17]:
def custom_collate(batch,args):
    # returns a [batch x seq x feats] tensor
    # feats: [padded_positions,positions,inputs_ts,items,users,targets,targets_ts]

    bs = len(batch)
    feat_len = len(batch[0])
    batch_seq = torch.zeros(bs,args.seq_len, feat_len, dtype=torch.long)
    for ib,b in enumerate(batch):
        for ifeat,feat in enumerate(b):
            batch_seq[ib,b[0],ifeat] = feat
    return batch_seq

In [18]:
class SequenceDataset(data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

def collate_fn_padd(batch):
    '''
    Padds batch of variable length

    note: it converts things ToTensor manually here since the ToTensor transform
    assume it takes in images rather than arbitrary tensors.
    '''
    ## get sequence lengths
    lengths = torch.tensor([ t.shape[0] for t in batch ]).to(device)
    ## padd
    batch = [ torch.Tensor(t).to(device) for t in batch ]
    batch = torch.nn.utils.rnn.pad_sequence(batch)
    ## compute mask
    mask = (batch != 0).to(device)
    return batch, lengths, mask

def get_sequences(_data, _p1, _p2, args, max_u=int(10e9)):
    data_list = []

    _data = _data[_data.stop<_p2].copy()
    
    grouped = _data.groupby('user')
    for user_id, group in tqdm(grouped):
        group = group.sort_values('start')
        group = group.tail(args.seq_len+1)
        if len(group)<2: continue

        group = group.reset_index(drop=True) 
        
        # Get last interaction
        last_el = group.tail(1)
        yt = last_el.start.values[0]
        group.drop(last_el.index,inplace=True)

        # avoid including train in test/validation
        if yt < _p1 or yt >= _p2: continue

        padlen = args.seq_len - len(group)

        # sequence input features
        positions  = torch.LongTensor(group.index.values)
        inputs_ts  = torch.LongTensor(group.start.values)
        items      = torch.LongTensor(group['streamer'].values)
        duration   = torch.LongTensor(group['duration'].values)
        users      = torch.LongTensor(group.user.values)
        bpad       = torch.LongTensor(group.index.values + padlen)
        diff_du    = torch.LongTensor(np.diff(np.array(inputs_ts[1:].tolist() + [last_el.start.values[0]])))

        # sequence output features
        targets    = torch.LongTensor(items[1:].tolist() + [last_el.streamer.values[0]])
        targets_ts = torch.LongTensor(inputs_ts[1:].tolist() + [last_el.start.values[0]])

        data_list.append([bpad,positions,inputs_ts,items,users,targets,targets_ts,duration])

        # stop if user limit is reached
        if len(data_list)>max_u: break

    return SequenceDataset(data_list)

In [19]:
cache_tr = os.path.join(args.cache_dir,"douyu_10k_min_dur_tr.p")
cache_te = os.path.join(args.cache_dir,"douyu_10k_min_dur_te.p")
cache_va = os.path.join(args.cache_dir,"douyu_10k_min_dur_val.p")

mu = int(10e9)

In [20]:
datalist_tr = get_sequences(data_fu,0,args.pivot_1,args,mu)
datalist_va = get_sequences(data_fu,args.pivot_1,args.pivot_2,args,mu)
datalist_te = get_sequences(data_fu,args.pivot_2,max_step,args,mu)
    
pickle.dump(datalist_te, open(cache_te, "wb"))
pickle.dump(datalist_tr, open(cache_tr, "wb"))
pickle.dump(datalist_va, open(cache_va, "wb"))

100%|██████████████████████████████████████████████████████████████████████████| 11610/11610 [00:04<00:00, 2331.05it/s]


In [21]:
train_loader = DataLoader(datalist_tr,batch_size=args.batch_size,
                              collate_fn=lambda x: custom_collate(x,args))
val_loader   = DataLoader(datalist_va,batch_size=args.batch_size,
                              collate_fn=lambda x: custom_collate(x,args))
test_loader  = DataLoader(datalist_te,batch_size=args.batch_size,
                              collate_fn=lambda x: custom_collate(x,args))

In [22]:
from eval import *
from data import *
from modify_duration_embedding_v1 import *

In [23]:
args.model='LiveRec_dur_emb_v1'
MPATH6,MODEL6 = get_model_type_new(args)
MPATH6,MODEL6
model6 = MODEL6(args).to(args.device)
optimizer = optim.Adam(model6.parameters(),lr=args.lr,weight_decay=args.l2)

best_val = 0.0
best_max = args.early_stop
best_cnt = best_max

In [ ]:
print("training...")
for epoch in range(args.num_epochs):
    loss_all = 0.0; loss_cnt = 0
    model6.train()
    for data in tqdm(train_loader):
        data = data.to(args.device)
        optimizer.zero_grad()
    
        loss = model6.train_step(data)
            
        loss_all += loss.item()
        loss_cnt += (data[:,:,5]!=0).sum()
        
#         loss1 = loss.detach_().requires_grad_(True)
#         loss1.backward()
        loss.backward()
        optimizer.step()
            
        if torch.isnan(loss):
            print("loss is nan !") 
    
    scores = compute_recall(model6, val_loader, args, maxit=500)
    print('Epoch: {:03d}, Loss: {:.5f}'.format(epoch, loss_all/loss_cnt))
    print_scores(scores)
    
    hall = scores['all']['h01']
    if hall>best_val:
        best_val = hall
        torch.save(model6.state_dict(), MPATH6)
        best_cnt = best_max
    else:
        best_cnt -= 1
        if best_cnt == 0:
            break
    
model6 = MODEL6(args).to(args.device)
model6.load_state_dict(torch.load(MPATH6))
    
scores = compute_recall(model6, test_loader, args)
print("Final score")
print("="*11)
print('Epoch: {:03d}, Loss: {:.5f}'.format(epoch, loss_all/loss_cnt))
print_scores(scores)
save_scores(scores,args)

training...


100%|████████████████████████████████████████████████████████████████████████████████| 114/114 [00:26<00:00,  4.34it/s]
17it [00:02,  7.93it/s]


Epoch: 000, Loss: 2.00647
all: h@1: 0.18914 h@5: 0.27944 h@10: 0.32825 ndcg@1: 0.18914 ndcg@5: 0.23617 ndcg@10: 0.25166
new: h@1: 0.00598 h@5: 0.03438 h@10: 0.06726 ndcg@1: 0.00598 ndcg@5: 0.01913 ndcg@10: 0.02949
rep: h@1: 0.31546 h@5: 0.44845 h@10: 0.50825 ndcg@1: 0.31546 ndcg@5: 0.38586 ndcg@10: 0.40489
ratio:  0.5918242830994509


100%|████████████████████████████████████████████████████████████████████████████████| 114/114 [00:25<00:00,  4.41it/s]
17it [00:02,  7.76it/s]


Epoch: 001, Loss: 1.55432
all: h@1: 0.13789 h@5: 0.28005 h@10: 0.33679 ndcg@1: 0.13789 ndcg@5: 0.21049 ndcg@10: 0.22896
new: h@1: 0.00897 h@5: 0.06876 h@10: 0.10613 ndcg@1: 0.00897 ndcg@5: 0.03747 ndcg@10: 0.04935
rep: h@1: 0.22680 h@5: 0.42577 h@10: 0.49588 ndcg@1: 0.22680 ndcg@5: 0.32982 ndcg@10: 0.35284
ratio:  0.5918242830994509


...

100%|████████████████████████████████████████████████████████████████████████████████| 114/114 [00:25<00:00,  4.41it/s]
17it [00:02,  7.78it/s]


Epoch: 023, Loss: 1.15335
all: h@1: 0.24405 h@5: 0.39597 h@10: 0.44539 ndcg@1: 0.24405 ndcg@5: 0.32605 ndcg@10: 0.34211
new: h@1: 0.01943 h@5: 0.10314 h@10: 0.16442 ndcg@1: 0.01943 ndcg@5: 0.06253 ndcg@10: 0.08243
rep: h@1: 0.39897 h@5: 0.59794 h@10: 0.63918 ndcg@1: 0.39897 ndcg@5: 0.50781 ndcg@10: 0.52120
ratio:  0.5918242830994509


100%|████████████████████████████████████████████████████████████████████████████████| 114/114 [00:26<00:00,  4.36it/s]
17it [00:02,  7.87it/s]


Epoch: 024, Loss: 1.15093
all: h@1: 0.25259 h@5: 0.40024 h@10: 0.44295 ndcg@1: 0.25259 ndcg@5: 0.33196 ndcg@10: 0.34590
new: h@1: 0.01943 h@5: 0.10613 h@10: 0.15994 ndcg@1: 0.01943 ndcg@5: 0.06368 ndcg@10: 0.08129
rep: h@1: 0.41340 h@5: 0.60309 h@10: 0.63814 ndcg@1: 0.41340 ndcg@5: 0.51698 ndcg@10: 0.52839
ratio:  0.5918242830994509


 89%|██████████████████████████████████████████████████████████████████████▉         | 101/114 [00:23<00:03,  3.96it/s]